# **La clase Pandas Fast**

Una clase que utiliza la librería `concurrent` para realizar cómputos utilizando `Threads`, extendiendo algunas funcionalidades de pandas. Actualmente están implementados: 

- `apply`
- `progress_apply`
- `to_csv`
- `to_pickle`
- `to_excel`
- `head`
- `tail`

___ 
## `PandasFast(df, processes = None)`
- Debe ser inicializada con un data frame `df`, el cual debe ser del tipo `pandas.core.DataFrame`. 
- Está la opción de definir la cantidad de `Threads` a utilizar, mediante la variable `processes`. Se le da este nombre a la variable en lugar de llamar `threads` o `n_threads` porque se siguen los mismos nombres que utilizan las funciones, en este caso, la variable `processes` es un input para la clase `ThreadPoolExecutor` que nos ayuda a realizar la computación de manera concurrente. Por default está inicializada como `None`, esto asignará $2$ `Threads`.
___
## **`apply` y `progress_apply`**
Recordemos que estas dos funciones hacen exactamente lo mismo, la única diferencia es que `progress_apply` utiliza internamente la librería `tqdm`, que nos permite tener una barra de progreso para tener un mayor control sobre el proceso.

**RECORDATORIO** Tanto `apply` como `progress_apply` nos permiten aplicar una función a cada componente de la (o las) columna(s) deseada(s). También se puede elegir hacerlo por filas, pero sólo están implementadas las funcionalidades por columnas.


En el caso de la clase `PandasFast`, estas funcionan de la misma forma, pues reciben: 

**Argumentos**
- `fun`: función que se quiere aplicar a una o más columnas, si tiene una variable de input, se asume que se aplica sobre una columna, si tiene dos, se aplica a dos columnas, y en el orden ingresado en la variable `columns`. Esta puede ser una función definida con `lambda` o definida afuera de la clase con `def`
- `columns`: un `str` con el nombre de una columna del  `df` o una lista de columnas. También puede ser una lista con el nombre de una sola columna, al igual que en pandas. **CUIDADO**, el la cantidad de columnas que deben ir en esta variable debe ser igual a la cantidad de inputs que recibe la función `fun`, y en el mismo orden en el que se solicita.

**Opcional**
- `new_columns`: un `str` o una lista con nombres de las nuevas columnas. Esto depende de la cantidad de salidas que entregue `fun`.
- `inplace`: si es `True`, la(s) nueva(s) columnas generadas serán incorporadas 

In [1]:
from PandasFast import PandasFast

___
# **Sección Ejemplos**

**Definiremos un dataframe para trabajar sobre el.**

In [2]:
import pandas as pd
from time import sleep
# ---------------------------------------------------------------------------
#   Experimento
# ---------------------------------------------------------------------------
df = pd.DataFrame(
    [[1,5,2,2],
     [2,5,3,1],
     [3,3,2,1],
     [4,1,4,4]]
    , columns = ['A','B','C','D'])
df

,A,B,C,D
0,1,5,2,2
1,2,5,3,1
2,3,3,2,1
3,4,1,4,4


**Inicializamos la clase PandasFast**

In [3]:
# Al inicializarla de esta forma, se utilizarán 2 Threads
pdfast = PandasFast(df)

## Ejemplo 1: función que actúa sobre una columna de `df`
Supongamos que queremos aplicar una función sobre una de las columnas de `df`, que espera 1 segundo y luego suma $1$ a cada fila. 

In [4]:
# Función de prueba
def fun(x):
    sleep(1)
    return x + 1

columns = "B" # o también puede llamarse como lista ["B"]

In [5]:
%%timeit
pdfast.apply(fun, columns)

2.02 s ± 8.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Si ahora queremos que nuestro output tenga un nombre específico, lo podemos indicar en la variable `new_columns`**

In [6]:
%%timeit
pdfast.apply(fun, columns, new_columns = "B_new")

2.03 s ± 8.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Y si queremos que nuestro output se agregue como nueva columna del df, lo podemos indicar seteando la variable `inplace=True`,**

In [7]:
%%timeit
pdfast.apply(fun, columns, new_columns = "B_new", inplace = True)

2.02 s ± 11.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**esto quedará guardado en el `df` dentro de la clase**

In [8]:
pdfast.df

,A,B,C,D,B_new
0,1,5,2,2,6
1,2,5,3,1,6
2,3,3,2,1,4
3,4,1,4,4,2


## **Puede replicar lo mismo con `progress_apply`, sólo se agregará una barra de progreso**

In [9]:
pdfast.progress_apply(fun, columns = ["A"])

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.00it/s]


,0
0,2
1,3
2,4
3,5


**IMPORTANTE**
Note que el proceso anterior demora $2$ segundos, eso es porque la clases se inicializó con `processes=None`, vale decir, con $2$ Threads por defecto. Si ahora la inicalizamos con $4$ Threads, debería demorar menos (pues son 4 filas sobre las cuales operar). Veamos

In [10]:
pdfast = PandasFast(df, processes=4) 

In [11]:
pdfast.progress_apply(fun, columns = ["A"])

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.97it/s]


,0
0,2
1,3
2,4
3,5


**En efecto, demora sólo $1$ segundo, pues cada fila es procesada en 1 Thread, y dado que la función toma 1 segundo aproximadamente en ejecutarse, se invierte sólo 1 segundo en lugar de 4, si lo hicieramos sólo usando el `progress_apply` de `pandas`. Veámoslo:**

In [12]:
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

df["A"].progress_apply(fun)

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


0    2
1    3
2    4
3    5
Name: A, dtype: int64

# PandasFast Victory!!!
___
___
## **Ahora, con una función que actúa sobre más de una columna y tiene un output**

In [13]:
def fun_col(x,y):
    sleep(1)
    return x + y

In [14]:
columns = ["A","B"]
new_columns = ["suma"]

In [15]:
pdfast.progress_apply(fun_col, columns, new_columns)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.00it/s]


,suma
0,6
1,7
2,6
3,5


## **Ahora, con una función que actúa sobre más de una columna y tiene dos output**

In [16]:
def fun_col_multioutput(x,y):
    sleep(1)
    return x + y, x-y

In [17]:
columns = ["A","B"]
new_columns = ["suma","resta"]

In [18]:
pdfast.progress_apply(fun_col_multioutput, columns, new_columns)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.95it/s]


,suma,resta
0,6,-4
1,7,-3
2,6,0
3,5,3


___
___
### **head() y tail()**
Funcionan exactamente igual que en pandas

In [19]:
pdfast.head(2)

,A,B,C,D,B_new
0,1,5,2,2,6
1,2,5,3,1,6


In [20]:
pdfast.tail(3)

,A,B,C,D,B_new
1,2,5,3,1,6
2,3,3,2,1,4
3,4,1,4,4,2


In [21]:
pdfast.df

,A,B,C,D,B_new
0,1,5,2,2,6
1,2,5,3,1,6
2,3,3,2,1,4
3,4,1,4,4,2


## **Guardar el `df`**

In [22]:
pdfast.to_csv("pdfast_example.csv")

# Un ejemplo más grande:
##  PandasFast vs Pandas

In [23]:
N = 10000
df = pd.DataFrame.from_dict(
    {'A': [1 for n in range(N)],
     'B': [2 for n in range(N)],
     'C': [3 for n in range(N)],
     'D': [4 for n in range(N)]}
)
df.shape

(10000, 4)

In [24]:
df.head()

,A,B,C,D
0,1,2,3,4
1,1,2,3,4
2,1,2,3,4
3,1,2,3,4
4,1,2,3,4


## Pandas y apply
Si tenemos un proceso donde cada evaluación toma aproximadamente 0.001 segundos en ejecutarse, tenemos los siguientes resultados.

In [25]:
# Función de prueba
def fun(x):
    sleep(0.001)
    return x*10

In [26]:
columns = "A"

In [27]:
res_pandas = df[columns].progress_apply(fun)

## PandasFast y apply
Esta vez, considerando que el df tiene muchas más filas que el primer ejemplo, vale la pena considerar mayor cantidad de *threads*, en este caso, usaré `30`

In [28]:
pdfast = PandasFast(df, processes = 30)
res_pandasfast = pdfast.progress_apply(fun, columns)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1934.01it/s]
